In [1]:
from __future__ import print_function
import json
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier,Pool,cv
from math import cos, asin, sqrt, pi

import seaborn as sns
import re
import sys
import seaborn
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
porter = PorterStemmer()


sns.set_style('darkgrid')
# from shapely.geometry import Point, Polygon

%matplotlib inline

ps = PorterStemmer()
def porter_stemmer(list_t):
    temp = []
    for i in list_t:
        temp.append(ps.stem(i))
    return temp

def clean_text(list_t):
    temp = []
    for text in list_t:
        # remove backslash-apostrophe 
        text = re.sub("\'", "", text) 
        # remove everything except alphabets 
        text = re.sub("[^a-zA-Z]"," ",text) 
        # remove whitespaces 
        text = ' '.join(text.split()) 
        # convert text to lowercase 
        text = text.lower() 
        temp.append(text)
    
    return temp

def clean_description(text):
    text = re.sub('<.*?>', '', text)
    text = re.sub('\w+_\w+', '', text)
    text = re.sub(r'[^\w]', ' ', text)
    text = text.lower()
    text = ' '.join(text.split())
    return text

from nltk.corpus import stopwords
stop_words=set(stopwords.words("english"))

def findOutlierIndex(data, threshold):
    z = np.abs(stats.zscore(data))
    index_outliers = np.where(z > threshold)
    return index_outliers

def stemSentence_porter(sentence):
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def token_plot_porter(data):
    for index,plot in enumerate(data['description']):
        data['description'][index] = stemSentence_porter(plot)
    return data

In [2]:
d = None  
data = None  
with zipfile.ZipFile("train.json.zip", "r") as z:
   for filename in z.namelist():    
      with z.open(filename) as f:  
         data = f.read()  
         d = json.loads(data.decode("utf-8"))
        
train_data = pd.DataFrame.from_dict(d)

d = None  
data = None  
with zipfile.ZipFile("test.json.zip", "r") as z:
   for filename in z.namelist():    
      with z.open(filename) as f:  
         data = f.read()  
         d = json.loads(data.decode("utf-8"))
        
test_data = pd.DataFrame.from_dict(d)

listing_id = test_data['listing_id']

price_front_percentile = np.percentile(train_data.price, 1)
price_end_percentile = np.percentile(train_data.price, 99)
train_data = train_data[(train_data['price'] < np.int(price_end_percentile)) & (train_data['price'] > np.int(price_front_percentile))]

latitude_front_percentile = np.percentile(train_data.latitude, 1)
latitude_end_percentile = np.percentile(train_data.latitude, 99)
train_data = train_data[(train_data['latitude'] < np.float(latitude_end_percentile)) & (train_data['latitude'] > np.float(latitude_front_percentile))]


longitude_front_percentile = np.percentile(train_data.longitude, 1)
longitude_end_percentile = np.percentile(train_data.longitude, 99)
train_data = train_data[(train_data['longitude'] < np.float(longitude_end_percentile)) & (train_data['longitude'] > np.float(longitude_front_percentile))]

In [3]:
subway = pd.read_csv("subway.csv")
def distance_for_twopoints(lat1, lon1, lat2, lon2):
    p = pi/180     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin...

vec_distance_for_two = np.vectorize(distance_for_twopoints)

def distance(location):
    distance = vec_distance_for_two(location[0],location[1],subway['Station Latitude'],subway['Station Longitude']);
    min_dis = min(distance)
    return min_dis

In [4]:
def features_extraction(data):
    data['features'] = data['features'].apply(lambda x : clean_text(x))

    data['features'] = data['features'].apply(porter_stemmer)

    dic = {}
    for i in range(len(data)):
        for j in data["features"][i]:
            if j in dic:
                dic[j]+=1
            else:
                dic.setdefault(j,1)

    dic = {key:val for key, val in dic.items() if val > 20000}
    keys = list(dic.keys())
    keys_for_class = keys

    for i in keys:
        data[i] = 0

    for index,words in enumerate(data['features']):
        for i in words:
            if i in keys:
                data[i][index] = 1
    return list(dic.keys()),data

def description_extraction(data):
    data['description'] = data['description'].apply(clean_description)

    for index, plot in enumerate(data['description']):
        temp = plot.split()
        temp_list = ''
        for i in temp:
            if i not in stop_words:
                temp_list = temp_list + ' ' + i
        data['description'][index] = temp_list
        
    dic = {}
    for i in data['description']:
        token_words=word_tokenize(i)
        for j in range(len(token_words)-1):
            temp = token_words[j] + ' ' +token_words[j+1]
            if temp not in dic:
                dic.update({temp:1})
            else:
                dic[temp] += 1
                
    features = list(data.axes[1][:])
    dic = dict((k, v) for k, v in dic.items() if v >= 13000 and v < 16000)
    dic_filtered = {k: v for k, v in dic.items() if k not in features}
    dic_filtered = {k: v for k, v in sorted(dic_filtered.items(), key=lambda item: item[1], reverse = True)}

    keys = list(dic_filtered.keys())
    for i in keys:
        data[i] = 0
        
    for index,sentence in enumerate(data['description']):
        token_words=sentence.split(' ')
        for i in range( len(token_words) - 1 ):
            temp = token_words[i] + ' ' + token_words[i+1]
            if temp in keys:
                data[temp][index] = 1
    return list(dic.keys()),data

dic,train_data = features_extraction(train_data)
b_dic,train_data = description_extraction(train_data)

C:\Users\123\anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\123\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\123\anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
def text_features_test(data, new_features, keys_for_class, keys):
    data['features'] = data['features'].apply(lambda x : clean_text(x))
    data['features'] = data['features'].apply(porter_stemmer)
    
    for i in new_features:
        data[i] = 0
        
    for index,words in enumerate(data['features']):
        for i in words:
            if i in keys_for_class:
                data[i][index] = 1
                
    data['description'] = data['description'].apply(clean_description)

    for index, plot in enumerate(data['description']):
        temp = plot.split()
        temp_list = ''
        for i in temp:
            if i not in stop_words:
                temp_list = temp_list + ' ' + i
        data['description'][index] = temp_list

    data = token_plot_porter(data)
    
    for index,sentence in enumerate(data['description']):
        token_words=sentence.split(' ')
        for i in range(len(token_words) - 1 ):
            temp = token_words[i] + ' ' + token_words[i+1]
            if temp in keys:
                data[temp][index] = 1
    return data
      
text_features = dic + b_dic
test_data = text_features_test(test_data, text_features, dic, b_dic)

C:\Users\123\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\123\anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\123\anaconda3\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\123\anaconda3\lib\site-packages\ipykern

In [6]:
y = train_data['interest_level']
del train_data['interest_level']
x = train_data.copy()

In [7]:
# Feature enginering
# Add more features to help prediction 
# part of function are from teammates 
def computeCoordinateX(lat, lon):
    return np.cos(lat) * np.cos(lon)
getx = np.vectorize(computeCoordinateX)
def computeCoordinateY(lat, lon):
    return np.cos(lat) * np.sin(lon)
gety = np.vectorize(computeCoordinateY)
def getPriceOverRooms(price, bathroom, bedroom):
    return price / (1 + bedroom + bathroom)
getRatio = np.vectorize(getPriceOverRooms)

subway = pd.read_csv("subway.csv")
def distance_for_twopoints(lat1, lon1, lat2, lon2):
    p = pi/180     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin...
vec_distance_for_two = np.vectorize(distance_for_twopoints)
def distance(location):
    distance = vec_distance_for_two(location[0],location[1],subway['Station Latitude'],subway['Station Longitude']);
    min_dis = min(distance)
    return min_dis

def feature_enginering(df , dfName):
    #transform lat and lon
    df['Coor_X'] = getx(df['latitude'], df['longitude'])
    df['Coor_Y'] = gety(df['latitude'], df['longitude'])
    #compute price over rooms (less the better)
    df['PriceOverRooms'] = getRatio(df['price'], df['bathrooms'], df['bedrooms'])
    
    df["num_photos"] = df["photos"].apply(len)
    
    df["num_features"] = df["features"].apply(len)
    
    df["num_description_words"] = df["description"].apply(lambda x: len(x.split(" ")))
    
    df["num_furniture"] =  df["bathrooms"] + df["bedrooms"] 
    
    df["created"] = pd.to_datetime(df["created"])
    
    df["created_month"] = df["created"].dt.month
    
    df["created_day"] = df["created"].dt.day
    
    df["created_hour"] = df["created"].dt.hour
    
    df["created_year"] = df["created"].dt.year
    
    df["total_days"] = (df["created_month"] -4.0)*30 + df["created_day"] +  df["created_hour"] /25.0
    
    df["diff_rank"]= df["total_days"]/df["listing_id"]
    
    df["price_latitue"] = (df["price"])/ (df["latitude"]+1.0)
    
    df["price_longtitude"] = (df["price"])/ (df["longitude"]-1.0) 
    
    df['location'] = df[['latitude','longitude']].values.tolist()
    
    df['distance'] = df['location'].apply(distance)
    
    return df

x = feature_enginering(x, "train")
test_data = feature_enginering(test_data, "test")

features_to_use = ["bathrooms", "bedrooms", "latitude", "longitude", "price" , 
                  "num_photos", "num_features", "num_description_words",
                   "created_month", "created_day", "created_hour" , "created_year", "Coor_X" , "Coor_Y"
                  , "PriceOverRooms" , "num_furniture" , "total_days" ,"diff_rank" , "price_latitue",
                  "price_longtitude"]
    

In [8]:
categorical = ["manager_id", "building_id"] + text_features
# text_features = ["description"]

In [9]:
x_ready = x[features_to_use + categorical]

test_data_ready = test_data[features_to_use + categorical]


## ！！！！！uncommon the flowing line for cross validation
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# x_ready.manager_id = le.fit_transform(x_ready.manager_id)
# x_ready.building_id = le.fit_transform(x_ready.building_id)

C:\Users\123\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [10]:
model = CatBoostClassifier(iterations=10000, 
                            task_type="GPU",
                            learning_rate=0.05,
                            depth=3,
                            loss_function='MultiClass',
#                            eval_metric='Accuracy',
#                            random_seed = 1,
#                            verbose = 200,
#                             random_strength=0.1,
#                             leaf_estimation_method='Newton',

#                             early_stopping_rounds = 500,
#                             verbose = 200,
#                            early_stopping_rounds = True,
                            devices='0:1')




model.fit(x_ready,
          y,
          verbose=False,
          cat_features=categorical,
#           text_features=text_features,
          use_best_model=False,
          plot=True)
# model.fit(train_dataset)
# Get predictions
result = model.predict_proba(test_data_ready)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [11]:
result

array([[1.50182474e-01, 2.81510584e-01, 5.68306941e-01],
       [1.13792271e-01, 7.65785694e-01, 1.20422035e-01],
       [2.06443531e-03, 9.81266233e-01, 1.66693313e-02],
       ...,
       [2.77490157e-04, 9.91662185e-01, 8.06032456e-03],
       [3.30160239e-03, 9.44005346e-01, 5.26930518e-02],
       [7.27482591e-02, 5.62430504e-01, 3.64821237e-01]])

In [12]:
aaa = model.predict(test_data_ready)
print('high', aaa[aaa=='high'].shape)
print('medium', aaa[aaa=='medium'].shape)

print('low', aaa[aaa=='low'].shape)

high (4110,)
medium (17899,)
low (52650,)


In [13]:
submit = {'listing_id':test_data['listing_id'], 'high': result[:,0], 'medium':result[:,2], 'low':result[:,1]}
pd.DataFrame(submit).to_csv('submission.csv',index = False)

In [14]:
# from sklearn.model_selection import cross_val_score

# scores = cross_val_score(model, x_ready, y, cv = 5)
# print(scores)

0:	learn: 1.0601258	total: 7.06ms	remaining: 1m 10s
1:	learn: 1.0263994	total: 13ms	remaining: 1m 4s
2:	learn: 0.9966065	total: 19ms	remaining: 1m 3s
3:	learn: 0.9700454	total: 24.8ms	remaining: 1m 1s
4:	learn: 0.9462621	total: 30.5ms	remaining: 1m
5:	learn: 0.9247330	total: 36.5ms	remaining: 1m
6:	learn: 0.9052489	total: 42ms	remaining: 1m
7:	learn: 0.8875913	total: 47.8ms	remaining: 59.7s
8:	learn: 0.8715207	total: 53.7ms	remaining: 59.7s
9:	learn: 0.8568664	total: 59.6ms	remaining: 59.6s
10:	learn: 0.8432857	total: 65.6ms	remaining: 59.6s
11:	learn: 0.8310444	total: 71.5ms	remaining: 59.5s
12:	learn: 0.8194671	total: 77.5ms	remaining: 59.5s
13:	learn: 0.8088450	total: 83.3ms	remaining: 59.4s
14:	learn: 0.7990054	total: 88.8ms	remaining: 59.1s
15:	learn: 0.7900044	total: 94.7ms	remaining: 59.1s
16:	learn: 0.7816409	total: 100ms	remaining: 58.9s
17:	learn: 0.7736446	total: 106ms	remaining: 58.8s
18:	learn: 0.7665451	total: 112ms	remaining: 58.7s
19:	learn: 0.7597257	total: 118ms	remai

181:	learn: 0.6134252	total: 1.1s	remaining: 59.2s
182:	learn: 0.6131688	total: 1.1s	remaining: 59.2s
183:	learn: 0.6130413	total: 1.11s	remaining: 59.1s
184:	learn: 0.6127842	total: 1.11s	remaining: 59.1s
185:	learn: 0.6126423	total: 1.12s	remaining: 59.1s
186:	learn: 0.6124939	total: 1.13s	remaining: 59.1s
187:	learn: 0.6123105	total: 1.13s	remaining: 59.1s
188:	learn: 0.6120874	total: 1.14s	remaining: 59.1s
189:	learn: 0.6119489	total: 1.14s	remaining: 59.1s
190:	learn: 0.6117511	total: 1.15s	remaining: 59s
191:	learn: 0.6115936	total: 1.15s	remaining: 59s
192:	learn: 0.6113824	total: 1.16s	remaining: 59s
193:	learn: 0.6112449	total: 1.17s	remaining: 59s
194:	learn: 0.6110509	total: 1.17s	remaining: 59s
195:	learn: 0.6108807	total: 1.18s	remaining: 59s
196:	learn: 0.6107135	total: 1.19s	remaining: 59s
197:	learn: 0.6105895	total: 1.19s	remaining: 59s
198:	learn: 0.6104557	total: 1.2s	remaining: 59s
199:	learn: 0.6103267	total: 1.2s	remaining: 59s
200:	learn: 0.6102187	total: 1.21s	r

358:	learn: 0.5911772	total: 2.18s	remaining: 58.5s
359:	learn: 0.5910960	total: 2.19s	remaining: 58.5s
360:	learn: 0.5910239	total: 2.19s	remaining: 58.5s
361:	learn: 0.5908896	total: 2.2s	remaining: 58.5s
362:	learn: 0.5907659	total: 2.2s	remaining: 58.5s
363:	learn: 0.5906979	total: 2.21s	remaining: 58.5s
364:	learn: 0.5905832	total: 2.22s	remaining: 58.5s
365:	learn: 0.5904692	total: 2.22s	remaining: 58.5s
366:	learn: 0.5903867	total: 2.23s	remaining: 58.5s
367:	learn: 0.5902394	total: 2.23s	remaining: 58.5s
368:	learn: 0.5901139	total: 2.24s	remaining: 58.5s
369:	learn: 0.5900268	total: 2.25s	remaining: 58.5s
370:	learn: 0.5899508	total: 2.25s	remaining: 58.5s
371:	learn: 0.5898900	total: 2.26s	remaining: 58.5s
372:	learn: 0.5898496	total: 2.27s	remaining: 58.5s
373:	learn: 0.5897611	total: 2.27s	remaining: 58.5s
374:	learn: 0.5896872	total: 2.28s	remaining: 58.5s
375:	learn: 0.5896308	total: 2.28s	remaining: 58.5s
376:	learn: 0.5895788	total: 2.29s	remaining: 58.4s
377:	learn: 0.

537:	learn: 0.5790306	total: 3.27s	remaining: 57.5s
538:	learn: 0.5789900	total: 3.27s	remaining: 57.4s
539:	learn: 0.5789613	total: 3.28s	remaining: 57.4s
540:	learn: 0.5789305	total: 3.28s	remaining: 57.4s
541:	learn: 0.5788816	total: 3.29s	remaining: 57.4s
542:	learn: 0.5788559	total: 3.3s	remaining: 57.4s
543:	learn: 0.5788126	total: 3.3s	remaining: 57.4s
544:	learn: 0.5787429	total: 3.31s	remaining: 57.4s
545:	learn: 0.5786670	total: 3.31s	remaining: 57.4s
546:	learn: 0.5785939	total: 3.32s	remaining: 57.4s
547:	learn: 0.5785301	total: 3.33s	remaining: 57.4s
548:	learn: 0.5784903	total: 3.33s	remaining: 57.3s
549:	learn: 0.5784159	total: 3.34s	remaining: 57.3s
550:	learn: 0.5783775	total: 3.35s	remaining: 57.4s
551:	learn: 0.5783033	total: 3.35s	remaining: 57.4s
552:	learn: 0.5782353	total: 3.36s	remaining: 57.4s
553:	learn: 0.5781830	total: 3.36s	remaining: 57.4s
554:	learn: 0.5781313	total: 3.37s	remaining: 57.4s
555:	learn: 0.5780871	total: 3.38s	remaining: 57.4s
556:	learn: 0.

716:	learn: 0.5707104	total: 4.35s	remaining: 56.3s
717:	learn: 0.5706767	total: 4.36s	remaining: 56.3s
718:	learn: 0.5706290	total: 4.36s	remaining: 56.3s
719:	learn: 0.5705792	total: 4.37s	remaining: 56.3s
720:	learn: 0.5705466	total: 4.38s	remaining: 56.3s
721:	learn: 0.5705097	total: 4.38s	remaining: 56.3s
722:	learn: 0.5704735	total: 4.39s	remaining: 56.3s
723:	learn: 0.5704341	total: 4.4s	remaining: 56.3s
724:	learn: 0.5704160	total: 4.4s	remaining: 56.3s
725:	learn: 0.5703699	total: 4.41s	remaining: 56.3s
726:	learn: 0.5703204	total: 4.41s	remaining: 56.3s
727:	learn: 0.5702747	total: 4.42s	remaining: 56.3s
728:	learn: 0.5702384	total: 4.43s	remaining: 56.3s
729:	learn: 0.5702018	total: 4.43s	remaining: 56.3s
730:	learn: 0.5701569	total: 4.44s	remaining: 56.3s
731:	learn: 0.5700826	total: 4.45s	remaining: 56.3s
732:	learn: 0.5700353	total: 4.45s	remaining: 56.3s
733:	learn: 0.5699882	total: 4.46s	remaining: 56.3s
734:	learn: 0.5699710	total: 4.46s	remaining: 56.3s
735:	learn: 0.

...

9244:	learn: 0.4412435	total: 56.3s	remaining: 4.6s
9245:	learn: 0.4412398	total: 56.4s	remaining: 4.59s
9246:	learn: 0.4412332	total: 56.4s	remaining: 4.59s
9247:	learn: 0.4412244	total: 56.4s	remaining: 4.58s
9248:	learn: 0.4412108	total: 56.4s	remaining: 4.58s
9249:	learn: 0.4412015	total: 56.4s	remaining: 4.57s
9250:	learn: 0.4411945	total: 56.4s	remaining: 4.56s
9251:	learn: 0.4411857	total: 56.4s	remaining: 4.56s
9252:	learn: 0.4411712	total: 56.4s	remaining: 4.55s
9253:	learn: 0.4411667	total: 56.4s	remaining: 4.55s
9254:	learn: 0.4411607	total: 56.4s	remaining: 4.54s
9255:	learn: 0.4411479	total: 56.4s	remaining: 4.53s
9256:	learn: 0.4411385	total: 56.4s	remaining: 4.53s
9257:	learn: 0.4411262	total: 56.4s	remaining: 4.52s
9258:	learn: 0.4411164	total: 56.4s	remaining: 4.52s
9259:	learn: 0.4411082	total: 56.4s	remaining: 4.51s
9260:	learn: 0.4411016	total: 56.4s	remaining: 4.5s
9261:	learn: 0.4410964	total: 56.4s	remaining: 4.5s
9262:	learn: 0.4410920	total: 56.5s	remaining: 4.

9425:	learn: 0.4396055	total: 57.4s	remaining: 3.5s
9426:	learn: 0.4395924	total: 57.4s	remaining: 3.49s
9427:	learn: 0.4395890	total: 57.4s	remaining: 3.48s
9428:	learn: 0.4395793	total: 57.5s	remaining: 3.48s
9429:	learn: 0.4395728	total: 57.5s	remaining: 3.47s
9430:	learn: 0.4395578	total: 57.5s	remaining: 3.47s
9431:	learn: 0.4395493	total: 57.5s	remaining: 3.46s
9432:	learn: 0.4395452	total: 57.5s	remaining: 3.45s
9433:	learn: 0.4395325	total: 57.5s	remaining: 3.45s
9434:	learn: 0.4395253	total: 57.5s	remaining: 3.44s
9435:	learn: 0.4395177	total: 57.5s	remaining: 3.44s
9436:	learn: 0.4395076	total: 57.5s	remaining: 3.43s
9437:	learn: 0.4395052	total: 57.5s	remaining: 3.42s
9438:	learn: 0.4394962	total: 57.5s	remaining: 3.42s
9439:	learn: 0.4394789	total: 57.5s	remaining: 3.41s
9440:	learn: 0.4394704	total: 57.5s	remaining: 3.4s
9441:	learn: 0.4394581	total: 57.5s	remaining: 3.4s
9442:	learn: 0.4394527	total: 57.5s	remaining: 3.39s
9443:	learn: 0.4394424	total: 57.5s	remaining: 3.

9608:	learn: 0.4381064	total: 58.5s	remaining: 2.38s
9609:	learn: 0.4381001	total: 58.5s	remaining: 2.38s
9610:	learn: 0.4380845	total: 58.5s	remaining: 2.37s
9611:	learn: 0.4380736	total: 58.5s	remaining: 2.36s
9612:	learn: 0.4380698	total: 58.5s	remaining: 2.36s
9613:	learn: 0.4380628	total: 58.6s	remaining: 2.35s
9614:	learn: 0.4380571	total: 58.6s	remaining: 2.34s
9615:	learn: 0.4380500	total: 58.6s	remaining: 2.34s
9616:	learn: 0.4380355	total: 58.6s	remaining: 2.33s
9617:	learn: 0.4380312	total: 58.6s	remaining: 2.33s
9618:	learn: 0.4380204	total: 58.6s	remaining: 2.32s
9619:	learn: 0.4380054	total: 58.6s	remaining: 2.31s
9620:	learn: 0.4379908	total: 58.6s	remaining: 2.31s
9621:	learn: 0.4379883	total: 58.6s	remaining: 2.3s
9622:	learn: 0.4379767	total: 58.6s	remaining: 2.3s
9623:	learn: 0.4379703	total: 58.6s	remaining: 2.29s
9624:	learn: 0.4379669	total: 58.6s	remaining: 2.28s
9625:	learn: 0.4379578	total: 58.6s	remaining: 2.28s
9626:	learn: 0.4379510	total: 58.6s	remaining: 2

9791:	learn: 0.4366051	total: 59.6s	remaining: 1.27s
9792:	learn: 0.4365947	total: 59.6s	remaining: 1.26s
9793:	learn: 0.4365870	total: 59.6s	remaining: 1.25s
9794:	learn: 0.4365762	total: 59.6s	remaining: 1.25s
9795:	learn: 0.4365657	total: 59.6s	remaining: 1.24s
9796:	learn: 0.4365589	total: 59.6s	remaining: 1.24s
9797:	learn: 0.4365501	total: 59.6s	remaining: 1.23s
9798:	learn: 0.4365462	total: 59.7s	remaining: 1.22s
9799:	learn: 0.4365320	total: 59.7s	remaining: 1.22s
9800:	learn: 0.4365246	total: 59.7s	remaining: 1.21s
9801:	learn: 0.4365202	total: 59.7s	remaining: 1.21s
9802:	learn: 0.4365065	total: 59.7s	remaining: 1.2s
9803:	learn: 0.4365006	total: 59.7s	remaining: 1.19s
9804:	learn: 0.4364912	total: 59.7s	remaining: 1.19s
9805:	learn: 0.4364806	total: 59.7s	remaining: 1.18s
9806:	learn: 0.4364720	total: 59.7s	remaining: 1.17s
9807:	learn: 0.4364677	total: 59.7s	remaining: 1.17s
9808:	learn: 0.4364613	total: 59.7s	remaining: 1.16s
9809:	learn: 0.4364527	total: 59.7s	remaining: 

9973:	learn: 0.4350430	total: 1m	remaining: 158ms
9974:	learn: 0.4350343	total: 1m	remaining: 152ms
9975:	learn: 0.4350256	total: 1m	remaining: 146ms
9976:	learn: 0.4350149	total: 1m	remaining: 140ms
9977:	learn: 0.4350036	total: 1m	remaining: 134ms
9978:	learn: 0.4349856	total: 1m	remaining: 128ms
9979:	learn: 0.4349767	total: 1m	remaining: 122ms
9980:	learn: 0.4349648	total: 1m	remaining: 116ms
9981:	learn: 0.4349564	total: 1m	remaining: 110ms
9982:	learn: 0.4349493	total: 1m	remaining: 103ms
9983:	learn: 0.4349439	total: 1m	remaining: 97.4ms
9984:	learn: 0.4349376	total: 1m	remaining: 91.3ms
9985:	learn: 0.4349258	total: 1m	remaining: 85.2ms
9986:	learn: 0.4349205	total: 1m	remaining: 79.1ms
9987:	learn: 0.4349099	total: 1m	remaining: 73ms
9988:	learn: 0.4349009	total: 1m	remaining: 66.9ms
9989:	learn: 0.4348882	total: 1m	remaining: 60.9ms
9990:	learn: 0.4348756	total: 1m	remaining: 54.8ms
9991:	learn: 0.4348687	total: 1m	remaining: 48.7ms
9992:	learn: 0.4348655	total: 1m	remaining:

In [16]:
# sum(scores)/len(scores)

7450067304741567

In [18]:
# from sklearn.metrics import f1_score
# from sklearn.model_selection import KFold, cross_val_score, train_test_split
# # X_train, X_test, y_train, y_test = train_test_split(x_ready, y, random_state=0)
# y_true = np.array(y)
# y_scores = np.array(model.predict(x_ready))
# f1_score(y_true, y_scores, average='micro')

0.7784137205041567